In [183]:
import pandas as pd
from collections import defaultdict
import numpy as np
import json
from collections import Counter
import nltk

In [184]:
train_data = []
test_data=[]
dev_data=[]
with open("data/train", 'r') as f:
    sentence_lines = f.readlines()    
    for line in sentence_lines:
        data = line.split()
        train_data.append(data)
with open("data/dev", 'r') as f:
    sentence_lines = f.readlines()
    for line in sentence_lines:
        data = line.split()
        dev_data.append(data)
with open("data/test", 'r') as f:
    sentence_lines = f.readlines()
    for line in sentence_lines:
        data = line.split()
        test_data.append(data)
        
train_data.append([])
test_data.append([])
dev_data.append([])

### Create Vocabulary

In [285]:
vocab = defaultdict(int)
for l in train_data:
    if l:
        vocab[l[1]] += 1    

for key in list(vocab.keys()):
    if vocab[key] < 2:
        vocab['< unk >'] += 1
        del vocab[key]

index=1
with open('vocab.txt', 'w') as f:
    f.write('< unk >'+'\t'+'0'+'\t'+str(vocab['< unk >'])+'\n')
    for w in sorted(vocab, key=vocab.get, reverse=True):
        if w != '< unk >':
            f.write(w+'\t'+str(index)+'\t'+str(vocab[w])+'\n')
            index += 1

print("Threshold considered = 2")
print("Total size of vocabulary = ", len(vocab.keys()))
print("Total number of < unk > = ", vocab['< unk >'])

Threshold considered = 2
Total size of vocabulary =  23183
Total number of < unk > =  20011


#### Compute emission and transion probabilities

In [300]:
## Add --s-- and --e-- to beggining and end of sentences

train_data_seq = []
temp=[('--s--', '--s--')]
for l in train_data:
    if not l:
        temp.append(('--e--', '--e--'))
        train_data_seq.append(temp[:])
        temp.clear()
        temp.append(('--s--', '--s--'))
    else:
        temp.append((l[1], l[2]))
            
dev_words_seq = []
dev_tags_seq = []
temp_words=['--s--']
temp_tags=['--s--']

for l in dev_data:
    if not l:
        temp_words.append('--e--')
        temp_tags.append('--e--')
        dev_words_seq.append(temp_words[:])
        dev_tags_seq.append(temp_tags[:])
        temp_words.clear()
        temp_tags.clear()
        temp_words.append('--s--')
        temp_tags.append('--s--')
    else:
        temp_words.append(l[1])
        temp_tags.append(l[2])
            
dev_tags_list = [element for innerList in dev_tags_seq for element in innerList]


test_data_seq = []
temp_test=['--s--']
for l in test_data:
    if not l:
        temp_test.append('--e--')
        test_data_seq.append(temp_test[:])
        temp_test.clear()
        temp_test.append('--s--')
    else:
        temp_test.append(l[1])            


In [301]:
emission_count = defaultdict(lambda: defaultdict(int))
emission_prob =defaultdict(lambda: defaultdict(int))
transition_count = defaultdict(lambda: defaultdict(int))
transition_prob = defaultdict(lambda: defaultdict(int))
tag_count = defaultdict(int)
tags_of_word = defaultdict(set)

vocab['--e--'] = 1
vocab['--s--'] = 1
for sentence in train_data_seq:
    for (w,t) in sentence:
        if w not in vocab:
            w='< unk >'
        emission_count[t][w]+=1
        tag_count[t] += 1
        tags_of_word[w].add(t)        
        
for tag in emission_count.keys():
    for word in emission_count[tag].keys():
        emission_prob[tag][word]=emission_count[tag][word]/tag_count[tag]
        
for sentence in train_data_seq:
    bigram=list(nltk.bigrams(sentence))
    for b1,b2 in bigram:
        transition_count[b1[1]][b2[1]]+=1

for tag in transition_count.keys():
    for next_tag in transition_count[tag].keys():
        transition_prob[tag][next_tag]=transition_count[tag][next_tag]/tag_count[tag]
        
transition_paramter_count=0
for key in transition_prob.keys():
    if (key == '--s--' or key == '--e--'):
        continue
    transition_paramter_count += len(transition_prob[key].keys())
    
emission_paramter_count=0
for key in emission_prob.keys():
    if (key == '--s--' or key == '--e--'):
        continue
    emission_paramter_count += len(emission_prob[key].keys())
    
print(f"Total Emission paramters = {emission_paramter_count}")
print(f"Total Transition paramters = {transition_paramter_count}")


Total Emission paramters = 30303
Total Transition paramters = 1375


In [188]:
#converting nested dictionary to dictionary with key as pair
def convert_transion():
    new_d = {}
    for k1 in transition_prob.keys():
         if k1 != '--e--':
            for k2 in transition_prob[k1].keys():
                if k2 != '--e--':
                    new_d[str('('+k1+','+k2+')')] = transition_prob[k1][k2]
                    
    return new_d

def convert_emission():
    new_d={}
    for k1 in emission_prob.keys():
         if k1 != '--s--':
            for k2 in emission_prob[k1].keys():
                if k2 != '--s--':
                    new_d[str('('+k1+','+k2+')')] = emission_prob[k1][k2]
                    
    return new_d
  
with open('hmm.json', 'w') as fp:
    json.dump(convert_transion(), fp,indent=2)
    json.dump(convert_emission(), fp,indent=2)

### Greedy Decoding

In [282]:
def findAccuracy(output_sequence, Y):
    correct=0
    total_predictions=0
    for p,t in zip(output_sequence, Y):
        if p==t:
            correct += 1
#     correct += sum(p == t for p, t in zip(output_sequence, Y))
    total_predictions = len(output_sequence)
    acc=correct / total_predictions
    return acc

In [190]:
def greedy_decoding(word_seq):  
    greedy_preds=[]
    for sentence in word_seq:
        greedy_preds.append('--s--')
        for key, word in enumerate(sentence[1:-1]):
            tag_max = ".";
            max_prob = 0;
            p = [] 
            for tag in tag_count.keys():
                if key == 0:
                    transition_p = transition_prob['--s--'][tag]
                else:
                    transition_p = transition_prob[greedy_preds[-1]][tag]

                tag_prob = emission_prob[tag][word] * transition_p  
                if(tag_prob > max_prob):
                    tag_max = tag
                    max_prob = tag_prob  
            greedy_preds.append(tag_max)
        greedy_preds.append('--e--')
        
    return greedy_preds

dev_preds_g = greedy_decoding(dev_words_seq)
test_preds_g = greedy_decoding(test_data_seq)

g_acc=findAccuracy(dev_preds_g, dev_tags_list)
print(f"Accuracy of Greedy Decoding algorithm = {g_acc}")




Accuracy of Greedy Decoding algorithm = 0.9169385668874543


In [191]:
t=[]
for pred_tag in test_preds_g:
    if pred_tag == '--s--':
        continue
    elif pred_tag == '--e--':
        t.append([])
    else:
        t.append(pred_tag)

with open('greedy.out', 'w') as f:
    for index, s in enumerate(test_data[:-1]):
        if s:
            f.write(s[0]+'\t'+s[1]+'\t'+t[index]+'\n')
        else:
            f.write('\n')     

### Viterbi

In [194]:
v=[]
for pred_tag in test_pred_v:
    if pred_tag == '--s--':
        continue
    elif pred_tag == '--e--':
        v.append([])
    else:
        v.append(pred_tag)

with open('viterbi.out', 'w') as f:
    for index, s in enumerate(test_data[:-1]):
        if s:
            f.write(s[0]+'\t'+s[1]+'\t'+v[index]+'\n')
        else:
            f.write('\n')

In [311]:
preds=[]
for x in range(len(dev_words_seq)):
    predicted_tags = defaultdict(lambda: defaultdict(list))
    sentence = dev_words_seq[x][:]
    for index, word in enumerate(sentence):
        word = word if word in vocab.keys() else '< unk >'
        possible_tags_for_words = list(tags_of_word[word]) 
        if index == 1:
            for tag in possible_tags_for_words:                
                if  emission_prob[tag][word] != 0:
                    predicted_tags[index][tag] = ['--s--',transition_prob['--s--'][tag]*emission_prob[tag][word]]
                else:
                    predicted_tags[index][tag]  = ['--s--', 0.00001]
                    
        elif index>=1:
            for tag in possible_tags_for_words:
                previous_tags = list(predicted_tags[index-1].keys())
                for prev_tag in previous_tags:                        
                    if emission_prob[tag][word] == 0:
                        temp = predicted_tags[index-1][prev_tag][1]*0.00001
                    else:
                        temp = predicted_tags[index-1][prev_tag][1]*transition_prob[prev_tag][tag]*emission_prob[tag][word]
                        
                    if  len(predicted_tags[index][tag])>0:  
                        if temp > predicted_tags[index][tag][1]:                        
                             predicted_tags[index][tag] = [prev_tag, temp]
                    else:
                        predicted_tags[index][tag] = [prev_tag, temp]
    
    cur_sentence_len = len(sentence)
    final_pred = []
    p_tag = ''
    for i in range(cur_sentence_len-1,0,-1):
        if i==cur_sentence_len-1:
            last_word_tags = list(predicted_tags[i].keys())            
            temp2 = -1000
            max_tag = "."
            for key in last_word_tags:
                if temp2 < predicted_tags[i][key][1]:
                    temp2 = predicted_tags[i][key][1]
                    max_tag = key
                    p_tag = predicted_tags[i][key][0]
            final_pred.append(max_tag)
            
        else:
            final_pred.append(prev_tag)
            prev_tag = predicted_tags[i][prev_tag][0]
    final_pred.append('--s--')
    final_pred.reverse()       
    preds.append(final_pred[:])
    
dev_preds_v = [element for innerList in preds for element in innerList]
v_acc=findAccuracy(dev_tags_list, dev_preds_v)                                    

v_acc

0.9519961910630015

In [ ]:
v=[]
for pred_tag in test_pred_v:
    if pred_tag == '--s--':
        continue
    elif pred_tag == '--e--':
        v.append([])
    else:
        v.append(pred_tag)

with open('viterbi.out', 'w') as f:
    for index, s in enumerate(test_data[:-1]):
        if s:
            f.write(s[0]+'\t'+s[1]+'\t'+v[index]+'\n')
        else:
            f.write('\n')